# Домашнее задание 2 
## Contrastive and non-contrastive methods in CXR images

## Оценивание

Задание должно быть выполнено самостоятельно. Похожие решения будут считаться плагиатом. Если вы опирались на внешний источник в реализации, необходимо указать ссылку на него. 

В качестве решения, необходимо предоставить код (`train.py` с аргументами для выбора датасета/метода) + отчет, в котором будут отображены все детали выбора гиперпараметров, комментарии, сопровождающие графики, а так же ответы на вопросы в ДЗ. Оформляйте отчет четко и читаемо. Плохо оформленный код, плохо оформленные графики негативно скажутся на оценке, так же как и неэффективная реализация.

## Введение

Вам предстоит реализовать (задание 0) и поработать с двумя методами - [SimCLR](https://arxiv.org/abs/2002.05709) и [VICReg](https://arxiv.org/abs/2105.04906). Обучать их будем на датасете, относящемся к домену медицинских изображений (задания 1-4). Подключим онлайн пробинг (задание 3), а так же сравним с трансфером с imagenet домена в этот мед домен (задание 4).

### Датасеты [MedMNIST+](https://medmnist.com/)

Будем использовать уже подготовленные сконвертированные из DICOM'ов картинки. MedMNIST включает в себя два релевантных для нас датасета с рентгеновскими снимками грудной клетки:


| MedMNIST2D     | Data Modality | Tasks (# Classes/Labels)           | # Samples | # Training | # Validation | # Test |
|----------------|---------------|------------------------------------|-----------|------------|--------------|--------|
| ChestMNIST     | Chest X-Ray   | Multi-Label (14), Binary-Class (2) | 112,120   | 78,468     | 11,219       | 22,433 |
| PneumoniaMNIST | Chest X-Ray   | Binary-Class (2)                   | 5,856     | 4,708      | 524          | 624    |

На этот раз будем использовать разрешение 224x224 (необходимо выставить `size` при инициализации датасета). Несколько картинок из ChestMNIST:

![CXR image examples from ChestMNIST](data/cxr.png)

## Задание 0 (2 балла)

Реализуйте SimCLR и VICReg на базе ResNet-18 энкодера. Для этого надо реализовать соответствующие лосс-функции и архитектуры проекционных голов. Убедитесь, в корректности реализации на CIFAR-10 (не забудьте применить коррекцию резнета для картинок разрешением 32x32 из предыдущего домашнего задания). Для этого, сначала сделайте предобучение на train части датасета в течении 100 эпох, затем сделайте линейный пробинг с замороженным выучившимся энкодером.

Референсный интервал top-1 accuracy для 100 эпох предобучения ~80-83% на линейном пробинге (если не получается, проверьте реализацию оптимизатора (**LARS**) и расписания шага обучения (`warmup_cosine`) или попробуйте подвигать learning rate).

**NB**
Чтобы сэкономить на психотерапевте, используйте оптимизатор [LARS](https://arxiv.org/abs/1708.03888) и `LinearWarmupCosineAnnealing` шедулер. Их нет в торче, но довольно просто реализовать самому или взять референсную реализацию.

In [1]:
from src.train import *

In [2]:
base_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
ssl_dataset = SSLDataset(base_dataset, get_ssl_transforms())
ssl_loader = DataLoader(ssl_dataset, batch_size=256, shuffle=True, num_workers=4)

model = SimCLR()
model = pretrain_ssl(model, ssl_loader, epochs=100)

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

linear_probe(model.encoder, train_loader, test_loader, epochs=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170M/170M [00:46<00:00, 3.67MB/s]


Epoch 1/100, Loss: 5.3763
Epoch 2/100, Loss: 5.1443
Epoch 3/100, Loss: 5.1092
Epoch 4/100, Loss: 5.0813
Epoch 5/100, Loss: 5.0561
Epoch 6/100, Loss: 5.0277
Epoch 7/100, Loss: 4.9996
Epoch 8/100, Loss: 4.9709
Epoch 9/100, Loss: 4.9426
Epoch 10/100, Loss: 4.9165
Epoch 11/100, Loss: 4.8947
Epoch 12/100, Loss: 4.8723
Epoch 13/100, Loss: 4.8490
Epoch 14/100, Loss: 4.8310
Epoch 15/100, Loss: 4.8129
Epoch 16/100, Loss: 4.7960
Epoch 17/100, Loss: 4.7817
Epoch 18/100, Loss: 4.7700
Epoch 19/100, Loss: 4.7592
Epoch 20/100, Loss: 4.7475
Epoch 21/100, Loss: 4.7393
Epoch 22/100, Loss: 4.7301
Epoch 23/100, Loss: 4.7213
Epoch 24/100, Loss: 4.7156
Epoch 25/100, Loss: 4.7105
Epoch 26/100, Loss: 4.7060
Epoch 27/100, Loss: 4.6992
Epoch 28/100, Loss: 4.6948
Epoch 29/100, Loss: 4.6907
Epoch 30/100, Loss: 4.6859
Epoch 31/100, Loss: 4.6828
Epoch 32/100, Loss: 4.6757
Epoch 33/100, Loss: 4.6735
Epoch 34/100, Loss: 4.6689
Epoch 35/100, Loss: 4.6627
Epoch 36/100, Loss: 4.6628
Epoch 37/100, Loss: 4.6598
Epoch 38/1

## Задание 1 (1 балл)

Загрузите упомянутые датасеты из `MedMNIST+` и проанализируйте данные. Например, посмотрите на количество и баланс классов, как устроена разметка по классам, найдите среднее и дисперсию значений пикселей. Определите **подходящие метрики и лосс** для конечной задачи для **каждого** из датасетов, аргументировано объясните ваш выбор.

Это задание выполнено в "src/EDA.ipynb"

## Задание 2 (2 балла)

CXR изображения выглядят специфично. Кажется, что нужно иметь и специфичные для таких картинок аугментации.
Поиграйтесь с трансформами и зафиксируйте набор, с которым вы будете проводить финальные запуски предобучения.

### Каким образом можно определить подходящие аугментации?

За неимением экспертного знания (если есть знакомый врач-рентгенолог, можно посоветоваться), будем отталкиваться от набора аугментаций в естественных картинках. Начнем с набора, используемого в SimCLR-подобных методах, для ImageNet. Примерно так готовый набор выглядит в `torchvision`'е (обратите внимание, что при создании `СolorJitter` указываются не сами интвервалы, а дельта, т.е. `brightness=0.4` дает `(0.6, 1.4)`):

```python
Compose(
      RandomResizedCrop(
          size=(224, 224),
          scale=(0.08, 1.0),
          ratio=(0.75, 1.3333333333333333),
          interpolation=InterpolationMode.BICUBIC,
          antialias=True)
      RandomApply(
          ColorJitter(
              brightness=(0.6, 1.4),
              contrast=(0.6, 1.4),
              saturation=(0.8, 1.2),
              hue=(-0.1, 0.1)))
      RandomGrayscale(p=0.2)
      GaussianBlur(p=0.5)
      Solarization(p=0.1)
      RandomHorizontalFlip(p=0.5)
      ToTensor()
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225], inplace=False)
```

Какие параметры аугментаций стоило бы поменять? Реализуйте набор трансформов, посмотрите какие картинки получаются на выходе (не забудьте перевести выход в нужный интервал значений для визуализации), поиграйтесь со значениями параметров (например, `scale` в `RandomResizedCrop` или `brightness` в `ColorJitter`). Какие трансформы стоит убрать? Попробуйте добавить инвертирование и повороты картинок на небольшой угол.

Рентгеновский снимок - это не отражение света, а проекция плотности тканей, поэтому дефолтный набор аугментаций будет работать плохо. 

## Задание 3 (4 балла)

Для того, чтобы честно найти подходящий набор аугментаций, надо проводить этап предобучения и затем оценивать качество получившихся репрезентаций на конечной задаче. Можно ли это как-то ускорить? Раз у нас есть разметка для всего датасета, воспользуемся ей и ускорим подбор аугментаций с помощью online probing'a.
Для этого, добавим голову для линейного пробинга `linear_probe` к нашему энкодеру (`backbone`) и проекционной голове (`projection_head`). Эта линейная "проба" будет состоять из одного линейного слоя из размерности выхода энкодера (например, 512 для ResNet18) в число классов на конечной задаче (например, 14 классов у ChestMNIST).

На каждой итерации **предобучения** будем учить `backbone` и `projection_head` на претекстовую задачу (например, SimCLR лосс), а линейную пробу на классификацию.
Получается такая двуглавая архитектура, где градиенты с претекстового лосса текут по проекционной голове и энкодеру, а градиенты с классификационного лосса только по линейной пробе (не забудьте правильно `detach'`нуться).

```
                      projection_head(h) -> ssl_loss
                    /
x -> encoder(x) -> h
                    \
                      probe(h.detach()) -> cls_loss
```

Записать это можно примерно так:
```python
for batch in aug_dataloader:
  x1, x2, y = batch
  h1, h2 = model.backbone(x1, x2)
  z1, z2 = model.projection_head(h1, h2)
  logits = model.linear_probe(h1.detach())
  total_loss = ssl_loss(z1, z2) + cls_loss(yhat, y)
  total_loss.backward()
```

Таким образом, мы сможем в реальном времени наблюдать за тем, как обучение на претекстовую задачу влияет на качество репрезентаций для конечной задачи. Конечно, это не то же самое, что провести полный цикл предобучения, а затем измерить качество на конечной задаче. Тем не менее, это обеспечивает быструю итерацию по конфигурациям гиперпараметров (например, выбор аугментаций). Можно делать запуски на небольшое число эпох и сравнивать онлайн метрики.

*NB* Если вспомнить STL-10 из ДЗ 1, разметка была доступна только для небольшого подмножества (`train` vs `unlabeled`). В таком случае онлайн пробинг все еще можно делать, пробу можно обучать во время валидационной эпохи на размеченном сплите (веса энкодера заморожены).


### Этап отбора аугментаций

Добавьте онлайн пробинг в пайплайн обучения SimCLR. Воспользуемся результатами онлайн пробинга на ранних эпохах для отбора аугментаций. Предложите свой набор аугментаций исходя из общих соображений и анализа из **задания 2**, так же можно попробовать [RandAugment](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.RandAugment.html). В качестве референсного набора зафиксируем следующую композицию трансформов:

```python
Compose(
      RandomApply(    
          RandomRotation(degrees=[-10.0, 10.0],
          interpolation=InterpolationMode.NEAREST,
          expand=False,
          fill=0))
      RandomResizedCrop(
          size=(224, 224),
          scale=(0.5, 1.0),
          ratio=(0.75, 1.3333333333333333),
          interpolation=InterpolationMode.BICUBIC, antialias=True)
      RandomApply(    
          ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2)))
      RandomHorizontalFlip(p=0.5)
      RandomApply(    
          Lambda(<lambda>, types=['object']))
      ToTensor()
      Normalize(mean=[0.5], std=[0.5], inplace=False)
),
```
где `<lambda>` это функция для инвертирования изображения.
Сравните выбранный вами набор аугментаций и референсный, какой из них лучше? Для сравнения можете ориентироваться на метрики онлайн пробинга на 5-10 эпохах.

### Этап полного предобучения

Зафиксируйте "лучший" набор и выполните полное предобучение (например, 50 эпох) с методами реализованными **задания 0**: SimCLR и VICReg (не забудьте использовать версию ResNet18 для разрешения 224х224).
После предобучения проведите (офлайн) линейный пробинг на всех датасетах (ChestMNIST и PneumoniaMNIST). В отчете продемонстрируйте графики обучения (значение лосса, значение метрик онлайн пробинга в ходе обучения), а также таблицу с финальными результатами. Проанализируйте разницу между SimCLR и VICReg.


Итого, краткий план задания:
1. Сформируйте собственный набор аугментаций для CXR и добавьте референсный.

2. Для каждого набора: предобучение SimCLR 5–10 эпох с онлайн-пробингом; сравниваем метрики.

3. Выбираем лучший набор → полное предобучение: SimCLR — 20+ эпох, VICReg — 20+ эпох.

4. Выполняем офлайн-линейный пробинг и сравниваем SimCLR и VICReg.

**Бонусный балл** получат решения, у которых значения финальных метрик соответсвуют supervised качеству (т.е. как если бы вы обучали ResNet-18 с нуля на каждом датасете). Значения метрик при supervised обучении можно найти [здесь](https://medmnist.com/). 

## Задание 4 (1 балл)

Попробуем начать предобучение не с рандомной инициализации, а с весов, полученных предобучением на естественных картинках. Предлагается два варианта на выбор (надо выбрать один):
* веса из библиотеки `torchvision`, которые были получены supervised обучением,
* веса из соответствующих чекпоинтов [solo-learn](https://github.com/vturrisi/solo-learn/tree/main), которые были получены self-supervised обучением на Imagenet-100 (100-классовая подвыборка ImageNet'а).

Для этого при создании энкодера в `torchvision.models.resnet` можно использовать параметр `weights` у `resnet18()`. 
После инициализации с предобученных весов, проведите такой же цикл предобучения из предыдущего пункта, и продемонстрируйте разницу в финальном качестве. Помогает или вредит старт с supervised imagenet'овских весов?